In [1]:
import numpy as np
import pandas as pd

# Required Functions
1. Need list or Database to take in Measured & Calculated values per droplet ionization
> Droplet Database: 
>> Idetifier: Droplet(i) for $i \in (1,2,3...)$ for a specific ionization<br>

>> Set:<br>
>> - V, b, g, p, $\eta_0$, $\rho$, d<br>

>> Measured/Computed:<br>
>> - $v_{y0i}$ = $\frac{ x_{i0}}{ t_{i0}}$<br>
>> - $a$<br>
>> - $\eta_{eff}$<br>
>> - $v_{yEi}$= $\frac{ x_{iE}}{ t_{iE}}$<br>
>> - $q$<br>


2. Need equation/function for radius (simplified for coding)
> $$a = \sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9v_{y0}\eta_0}{2 \rho g}} -\left(\frac{b}{2p}\right)$$
> > $$a = \sqrt{\left(\alpha\right)^2 + \beta} -\left(\alpha\right)$$


3. Need equation/function for charge (simplified for coding)
> $$ q = \frac{4}{3}\pi \rho g \frac{d}{V}\left(
\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9v_{y0}\eta_0}{2 \rho g}} -\left(\frac{b}{2p}\right)
\right)^3 \cdot \frac{v_{yE}-v_{y0}}{v_{y0}}$$
> > $$ q = \frac{4}{3}\pi \rho g \frac{d}{V}a^3 \cdot \frac{v_{yE}-v_{y0}}{v_{y0}}$$

4. Need equations for Error propagation 
5. Need Graphs see images saved from speaking with the professor 

In [2]:
# Constants 

b   = 8.20E-3         # Viscostiy Correction Factor (given)
g   = 9.80            # Gravity (given)
p   = 1.013E5         # Atmospheric Pressure (at room temp? - given in docs)
n_o = 1.824E-5        # Tabulated Viscosity/ Viscocity of air (given)
rho1 = 8.86E2         # Density of oil droplet (given)
rho2 = 0              # To be determined for accuracy   

#Stored List 
Constants = [b, g, p, n_o,rho1 ,rho2]

In [3]:
# Separation of plates (d) 
d   = 7.752E-3      # meters

# Error in Separation (?)

## Functions for $a$ 

> $$a = \sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9v_{y0}\eta_0}{2 \rho g}} -\left(\frac{b}{2p}\right)$$
> > $$a = \sqrt{\left(\alpha\right)^2 + \beta} -\left(\alpha\right)$$

In [4]:
# Fuction for radius a 
# assuming b, p, n_o, g are all set 
# inputs (x,t) for E = 0 

def radius(v_y0):     
    # Function: Takes (positive) velocity returns radius
    # Components of a 
    # terminal velocity with E = 0 
    
    # convert mm to m 
    v_y0 = v_y0*10**(-3)
    
    # alpha 
    alpha = b/(2*p)

    #beta
    beta = (9*v_y0*n_o)/(2*rho1*g)
    
    #radius a 
    a = np.sqrt((alpha**2)+beta) - alpha
   
    return a

## Function for charge $q$

- $q_1 = q_2=  q$ They are just expressed differently 

> $$ q = \frac{4}{3}\pi \rho g \frac{d}{V}\left(
\sqrt{\left(\frac{b}{2p}\right)^2 + \frac{9v_{y0}\eta_0}{2 \rho g}} -\left(\frac{b}{2p}\right)
\right)^3 \cdot \frac{v_{yE}-v_{y0}}{v_{y0}}$$
> > $$ q_1 = \frac{4}{3}\pi \rho g \frac{d}{V}a^3 \cdot \frac{v_{yE}-v_{y0}}{v_{y0}}$$

>>$$ q_2 = 18 \pi d \left( \frac{\eta_0^3}{2g\rho}\right)^{\frac{1}{2}}\cdot 
\left( \frac{1}{(1+\frac{b}{pa})}\right)^{\frac{3}{2}} \sqrt{v_{y0}}\cdot
\frac{(v_{yE}-v_{y0})}{V}
$$

In [5]:
# q1 - opting to just use one equation

def q1(a,v_y0,v_yE,V):
     # Funtion: Takes radius,Velocity E=0 and Velocity E != 0 
    #velocity: v_y0 E=0 
    #velocity: v_yE E != 0

    # convert mm to m 
    v_y0 = (v_y0)*10**(-3)
    v_yE = -(v_yE)*10**(-3) 


    #left term (lt) 
    lt = (4/3)*np.pi*rho1*g*(d/V)*a**3

    #right term (rt)
    rt = (v_yE - v_y0)/ v_y0

    q1 = lt*rt
    
    return q1

## Function for Effective Viscosity $\eta_{eff}(a)$ - if needed for paper

$$\eta_{eff}(a) = \frac{\eta_0}{1 + \frac{b}{pa}}
$$

In [6]:
# effective viscosity 

def neff(a): 
    n_effa = n_o/(1 + (b/(p*a)))

    return n_effa

# Function to pull data and create a list - per droplet

- Droplet_Values(Vf1,Vr1, Vf2,Vr2,Vf3,Vr3,vf av, vr av, Volts V)
- RETURN: list of selected drop

In [7]:

def Droplet_Values(n_drop, n_ionized, csv_name_as_string):
    # Funtion: Choose Droplet, Choose ionization, Call "file.csv"
    # Return a list of paramerters per droplet selected 
    # E.g.: droplet1_ionization2 = [velocities, Voltage]


    # Call the CSV file
    droplet = pd.read_csv(csv_name_as_string)

    # Select the specified droplet by droplet # and Ionization (rows)
    d1 = droplet[droplet['Droplet'].isin([n_drop]) & droplet['Ionized'].isin([n_ionized])]

    # Select the specified parameters (Velocities and Volts) (columns)
    d1 = d1[["Vf1","Vr1","Vf2","Vr2","Vf3","Vr3","vf av","vr av","Volts V"]]

    # Convert to array, already is a float
    d1 = d1.to_numpy()

    # Assign a list 
    d1_values = []

    # use a for loop to unpack the 0th element into our desired list

    for value in d1[0]:
        d1_values.append(value)

    return d1_values


# Function to pull ALL droplets and place them in a list of lists 
> Important, Each Droplet and Ionizatoin requires an input no empty rows

In [8]:
def CreateDropsDatabase(csv_file_name_as_string): # "filename.csv"
    # Function: Takes "file.csv" Returns list of lists for our drops
    # dependent on the function Droplet_Values()
    # E.g.: Drops = [d1[velocities,Voltage],...dn[velocities,Voltage]]
    # Full detial of function found in Oil-Drop.ipynb

    drops = []
   
    droplet_number = 1 
   
    while droplet_number is not None: 

        ionization_number = 0

       
        while ionization_number is not None: 
         

            try:
                d = Droplet_Values(droplet_number, ionization_number, csv_file_name_as_string)
        
                drops.append(d)
                ionization_number += 1

            except IndexError:
               
                if ionization_number == 0: 
                   
                    droplet_number = None
                    break
                else:
                    
                    ionization_number = None
        
        if droplet_number is not None:
            droplet_number += 1
    return drops 

In [11]:
all_drops = CreateDropsDatabase("Velocities.csv")
print(len(drops))
for x in drops: 
    print(x)

23
[3.65e-05, 0.00025, 4.72e-05, 0.000263, 3.7e-05, 0.000132, 4.02e-05, 0.000215, 501.0]
[0.000119, 4.31e-05, 0.000116, 4.39e-05, 0.000114, 0.0002, 0.000116, 9.57e-05, 501.0]
[0.000125, 0.000179, 0.000114, 0.000208, 0.000125, 0.0002, 0.000121, 0.000196, 501.0]
[8.77e-05, 0.000556, 0.000125, 0.000625, 0.000114, 0.000556, 0.000109, 0.000579, 501.0]
[0.000111, 0.000714, 0.000119, 0.000833, 0.000122, 0.000833, 0.000117, 0.000794, 501.0]
[3.76e-05, 0.000185, 3.23e-05, 0.000217, 4.24e-05, 0.000172, 3.74e-05, 0.000192, 501.0]
[3.01e-05, 0.000417, 2.7e-05, 0.000333, 2.44e-05, 0.000357, 2.72e-05, 0.000369, 501.0]
[2.79e-05, 0.000357, 3.31e-05, 0.000263, 2.92e-05, 0.000313, 3.01e-05, 0.000311, 501.0]
[4.55e-05, 0.000417, 2.98e-05, 0.000179, 2.53e-05, 0.000185, 3.35e-05, 0.00026, 501.0]
[4.55e-05, 0.000263, 3.62e-05, 0.00025, 2.33e-05, 0.00025, 3.5e-05, 0.000254, 501.0]
[2.58e-05, 0.00025, 2.36e-05, 0.000227, 3.03e-05, 0.000313, 2.66e-05, 0.000263, 501.0]
[5.56e-05, 0.000166, 5.58e-05, 0.000126, 

In [15]:
# just add as you go - requires looking at the csv file 
drops_no_ionization = [
    all_drops[0],all_drops[1],all_drops[5],all_drops[6],all_drops[7],all_drops[8],
    all_drops[9],all_drops[10],all_drops[11],all_drops[12],all_drops[13],
    all_drops[14],all_drops[17],all_drops[20],all_drops[21]
    ]

print(len(drops_no_ionization))
for x in drops_no_ionization: 
    print(x)

15
[3.65e-05, 0.00025, 4.72e-05, 0.000263, 3.7e-05, 0.000132, 4.02e-05, 0.000215, 501.0]
[0.000119, 4.31e-05, 0.000116, 4.39e-05, 0.000114, 0.0002, 0.000116, 9.57e-05, 501.0]
[3.76e-05, 0.000185, 3.23e-05, 0.000217, 4.24e-05, 0.000172, 3.74e-05, 0.000192, 501.0]
[3.01e-05, 0.000417, 2.7e-05, 0.000333, 2.44e-05, 0.000357, 2.72e-05, 0.000369, 501.0]
[2.79e-05, 0.000357, 3.31e-05, 0.000263, 2.92e-05, 0.000313, 3.01e-05, 0.000311, 501.0]
[4.55e-05, 0.000417, 2.98e-05, 0.000179, 2.53e-05, 0.000185, 3.35e-05, 0.00026, 501.0]
[4.55e-05, 0.000263, 3.62e-05, 0.00025, 2.33e-05, 0.00025, 3.5e-05, 0.000254, 501.0]
[2.58e-05, 0.00025, 2.36e-05, 0.000227, 3.03e-05, 0.000313, 2.66e-05, 0.000263, 501.0]
[5.56e-05, 0.000166, 5.58e-05, 0.000126, 5.57e-05, 0.000143, 5.57e-05, 0.000145, 501.0]
[4.31e-05, 0.000313, 4.72e-05, 0.000313, 4.03e-05, 0.000294, 4.35e-05, 0.000306, 501.0]
[4.65e-05, 0.000179, 5.32e-05, 0.000167, 4.85e-05, 0.000294, 4.94e-05, 0.000213, 501.0]
[3.95e-05, 0.00128, 4.09e-05, 0.000459,

In [ ]:
def Radii_Charge(drops): 
    # droplet = to loop through
    # drops a list of lists previously created

    # function takes list to loop through
    # returns two lists one for radius and one for charge 
    list_of_radius_values = []
    list_of_charge_values =[]


    for droplet in drops: 
        n = 0
        # print('new droplet')
        for i in range(3): 
            if n > 4: 
                break
            else: 
                if droplet[n] ==0 or droplet[n+1] ==0: 
                    list_of_radius_values.append(0)
                    list_of_charge_values.append(0)
                    # print("no value vf.vr")
                    continue
                else:
                    # print(n)
                    # print('new drop vf.vr')
                    # print(droplet[n])
                    # print(droplet[n+1])

                    ri = radius(droplet[n])
                    list_of_radius_values.append(ri)

                    #calculate charge 
                    qi = q1(ri,droplet[n],droplet[n+1],droplet[6])
                    list_of_charge_values.append(qi)
                    n += 2
    return list_of_radius_values, list_of_charge_values